In [1]:
import pandas as pd
import numpy as np

In [2]:
#read in data
df = pd.read_csv('Life_Expectancy_Data.csv',sep=r'\s*,\s*', engine='python')

df.describe()


,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,2938.000000,2928.000000,2928.000000,2938.000000,2744.000000,2938.000000,2385.000000,2938.000000,2904.000000,2938.000000,2919.000000,2712.00000,2919.000000,2938.000000,2490.000000,2.286000e+03,2904.000000,2904.000000,2771.000000,2775.000000
mean,2007.518720,69.224932,164.796448,30.303948,4.602861,738.251295,80.940461,2419.592240,38.321247,42.035739,82.550188,5.93819,82.324084,1.742103,7483.158469,1.275338e+07,4.839704,4.870317,0.627551,11.992793
std,4.613841,9.523867,124.292079,117.926501,4.052413,1987.914858,25.070016,11467.272489,20.044034,160.445548,23.428046,2.49832,23.716912,5.077785,14270.169342,6.101210e+07,4.420195,4.508882,0.210904,3.358920
min,2000.000000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.37000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2004.000000,63.100000,74.000000,0.000000,0.877500,4.685343,77.000000,0.000000,19.300000,0.000000,78.000000,4.26000,78.000000,0.100000,463.935626,1.957932e+05,1.600000,1.500000,0.493000,10.100000
50%,2008.000000,72.100000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.75500,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000
75%,2012.000000,75.700000,228.000000,22.000000,7.702500,441.534144,97.000000,360.250000,56.200000,28.000000,97.000000,7.49250,97.000000,0.800000,5910.806335,7.420359e+06,7.200000,7.200000,0.779000,14.300000
max,2015.000000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.60000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000


In [17]:
#remove categorical data
df = df._get_numeric_data()

#Deal with missing data
df = df[df['Life expectancy'].notna()] #drop rows without life expectancy value
df= df.fillna(df.mean())# impute other columns with the columns mean



#create numpy matrices
X = df.drop('Life expectancy',axis=1).to_numpy()
X=np.append(np.ones((X.shape[0],1)),X,axis=1) #add column of 1s
y = df['Life expectancy'].to_numpy()


#initialising weights
theta = np.zeros(X.shape[1])
n=df.shape[0] #number of training examples
d=df.shape[1] #number of features

#parameters
epsilon = 0.1
learning_rate = 0.1

20


In [4]:
df.describe()

,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,2928.00000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2.928000e+03,2928.000000,2928.000000,2928.000000,2928.000000
mean,2007.50000,69.224932,164.796448,30.407445,4.614856,740.321185,80.960842,2427.855874,38.235394,42.179303,82.548298,5.930163,82.321416,1.747712,7494.210719,1.276454e+07,4.850622,4.881423,0.627419,11.999639
std,4.61056,9.523867,124.292079,118.114450,3.914923,1990.930605,22.531361,11485.970937,19.850184,160.700547,23.340548,2.385478,23.629576,5.085542,13157.128331,5.390628e+07,4.396597,4.484890,0.205131,3.253691
min,2000.00000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.370000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2003.75000,63.100000,74.000000,0.000000,1.107500,4.853964,80.960842,0.000000,19.400000,0.000000,78.000000,4.370000,78.000000,0.100000,578.797095,4.181205e+05,1.600000,1.600000,0.504000,10.300000
50%,2007.50000,72.100000,144.000000,3.000000,4.160000,65.611455,87.000000,17.000000,43.000000,4.000000,93.000000,5.930163,93.000000,0.100000,3112.560501,3.640009e+06,3.400000,3.400000,0.662000,12.100000
75%,2011.25000,75.700000,228.000000,22.000000,7.400000,442.614322,96.000000,362.250000,56.100000,28.000000,97.000000,7.330000,97.000000,0.800000,7494.210719,1.276454e+07,7.100000,7.200000,0.773000,14.100000
max,2015.00000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,77.600000,2500.000000,99.000000,17.600000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000


In [5]:
print(X)
print(y)

[[1.000e+00 2.015e+03 2.630e+02 ... 1.730e+01 4.790e-01 1.010e+01]
 [1.000e+00 2.014e+03 2.710e+02 ... 1.750e+01 4.760e-01 1.000e+01]
 [1.000e+00 2.013e+03 2.680e+02 ... 1.770e+01 4.700e-01 9.900e+00]
 ...
 [1.000e+00 2.002e+03 7.300e+01 ... 1.300e+00 4.270e-01 1.000e+01]
 [1.000e+00 2.001e+03 6.860e+02 ... 1.700e+00 4.270e-01 9.800e+00]
 [1.000e+00 2.000e+03 6.650e+02 ... 1.120e+01 4.340e-01 9.800e+00]]
[65.  59.9 59.9 ... 44.8 45.3 46. ]


In [19]:
def cost(X,y,theta):
    h= np.dot(X,theta) #don't need to transpose theta as in numpy row vectors and column vectors are the same
    return (1/(2*n))*np.sum(np.square(h-y))

In [20]:
print(cost(X,y,theta))

2441.382120901639


In [73]:
def gradient_descent(X,y,theta,alpha,epsilon):
    n= X.shape[0]
    j = []
    j.append(cost(X,y,theta))
    for i in range(3):
        
        h=np.dot(X,theta)
        print(np.dot((h-y),X))
        print((alpha/n)*np.dot((h-y),X))
        theta= theta - (alpha/n)*(np.dot((h-y),X))
        #print(theta)
        j.append(cost(X,y,theta))
        
    print(j)
    
    
    
    

In [74]:
gradient_descent(X,y,theta,0.01,0.01)

[-2.02690600e+05 -4.06923233e+08 -3.09899377e+07 -5.51611900e+06
 -9.78214166e+05 -1.71249531e+08 -1.65384881e+07 -4.41646628e+08
 -8.06118890e+06 -7.55247430e+06 -1.70327492e+07 -1.21592819e+06
 -1.69996128e+07 -2.75343760e+05 -1.67692034e+09 -2.55775603e+12
 -9.25231141e+05 -9.31004200e+05 -1.31132489e+05 -2.49739304e+06]
[-6.92249317e-01 -1.38976514e+03 -1.05839951e+02 -1.88392042e+01
 -3.34089537e+00 -5.84868616e+02 -5.64839076e+01 -1.50835597e+03
 -2.75313829e+01 -2.57939696e+01 -5.81719578e+01 -4.15276022e+00
 -5.80587869e+01 -9.40381694e-01 -5.72718695e+03 -8.73550558e+06
 -3.15994242e+00 -3.17965915e+00 -4.47856862e-01 -8.52934783e+00]
[3.26485983e+17 6.55517080e+20 5.16608107e+19 9.92308650e+19
 1.33976445e+18 1.73914399e+20 2.30018334e+19 4.53260276e+21
 1.07532043e+19 1.32468757e+20 2.58201393e+19 1.71664232e+18
 2.60397810e+19 3.78640506e+17 1.98133244e+21 7.84675661e+25
 3.01524980e+18 3.04055523e+18 2.02613675e+17 3.78385049e+18]
[1.11504776e+12 2.23878784e+15 1.76437195e